<a href="https://colab.research.google.com/github/fhenri/FinanceAnalysis/blob/main/GenAI_Assistant_Stock_Market_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [2]:
## Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display, Markdown
from google.colab import userdata

import yfinance as yf


In [5]:
# Function to calculate start and end dates based on the number of years from today
def calculate_date_range(years):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=years * 365)
    return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

In [6]:
# Calculates and returns key performance indicators (RSI, Bollinger Bands, P/E Ratio, Beta, and MACD) for a list of stock tickers over a specified date range.
def calculate_kpis(tickers, start_date, end_date):
    """
    Calculate KPIs for a list of stocks over a given time period.

    Parameters:
    tickers (list): A list of stock ticker symbols.
    start_date (str): The start date for the analysis.
    end_date (str): The end date for the analysis.

    Returns:
    dict: A dictionary containing the KPIs for each stock.
    """
    kpi_data = {}
    for ticker in tickers:

        # Download historical stock data
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date)
        kpi_data[ticker] = {}

        # get Simple Moving Average 50 and 200 periods
        sma50 = data['Close'].rolling(window=50).mean()
        sma200 = data['Close'].rolling(window=200).mean()
        kpi_data[ticker]['SMA50'] = sma50
        kpi_data[ticker]['SMA200'] = sma200

        # Calculate RSI
        delta = data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()      # Average gains
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()     # Average losses
        rs = gain / loss                                                  # Relative strength
        rsi = 100 - (100 / (1 + rs))                                      # RSI formula
        kpi_data[ticker]['RSI'] = rsi

        # Calculate Bollinger Bands
        middle_band = data['Close'].rolling(window=20).mean()                     # Middle band (SMA)
        upper_band = middle_band + 2 * data['Close'].rolling(window=20).std()     # Upper band
        lower_band = middle_band - 2 * data['Close'].rolling(window=20).std()     # Lower band
        kpi_data[ticker]['Bollinger Bands'] = {
            'Middle Band': middle_band,
            'Upper Band': upper_band,
            'Lower Band': lower_band
        }

        # Calculate P/E Ratio
        try:
            eps = stock.info.get('trailingEps') # Get trailing EPS
            if eps and eps != 0:
                pe_ratio = data['Close'] / eps
                kpi_data[ticker]['P/E Ratio'] = pe_ratio # Calculate P/E ratio
            else:
                kpi_data[ticker]['P/E Ratio'] = None
        except Exception as e:
            kpi_data[ticker]['P/E Ratio'] = None
            print(f"An error occurred with ticker {ticker} P/E Ratio: {e}")

        # Calculate Beta
        try:
            beta = stock.info.get('beta') # Get beta value
            kpi_data[ticker]['Beta'] = beta
        except Exception as e:
            kpi_data[ticker]['Beta'] = None
            print(f"An error occurred with ticker {ticker} Beta: {e}")

        # Calculate MACD
        ema_12 = data['Close'].ewm(span=12, adjust=False).mean()    # 12-day EMA
        ema_26 = data['Close'].ewm(span=26, adjust=False).mean()    # 26-day EMA
        macd = ema_12 - ema_26    # MACD line
        signal = macd.ewm(span=9, adjust=False).mean()   # Signal line
        histogram = macd - signal    # MACD histogram
        kpi_data[ticker]['MACD'] = {
            'MACD': macd,
            'Signal Line': signal,
            'MACD Histogram': histogram
        }

    return kpi_data



In [7]:
# Define the number of years for the analysis
years = 2

# Calculate the start and end dates
start_date, end_date = calculate_date_range(years)

# List of assets to analyze
tickers = ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA']

In [8]:
# Calculate KPIs for the defined tickers and date range
kpi_data = calculate_kpis(tickers, start_date, end_date)

# Create a prompt to generate an executive summary with recommendations based on KPI data
prompt = f""" Read this data {kpi_data} and provide an executive summary with recommendations for each asset. This should be presented with 2 tables format.
The 1st table will recap the KPIs elements (in row the assets, and KPIs in columns with their value and how to interpret).
In the 2nd table you will give the following information:
- The current trend (uptrend, downtrend or range) for range indicate the lower and upper boundaries (zone price)
- The recommendation
- The confidence in the recommendation
- The elements of the recommendation (if any)
"""


<ipython-input-4-d59b134f3e63>:13: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt)


To provide a comprehensive executive summary with recommendations for each asset, we will present the data in two tables. The first table will recap the key performance indicators (KPIs) for each asset, and the second table will provide a detailed analysis including the current trend, recommendation, confidence level, and elements supporting the recommendation.

### Table 1: KPIs Recap

| Asset | SMA50 | SMA200 | RSI | Bollinger Bands | P/E Ratio | Beta | MACD |
|-------|-------|--------|-----|-----------------|-----------|------|------|
| AAPL  | 229.93 (Bullish) | 206.62 (Bullish) | 82.96 (Overbought) | Upper: 244.58, Middle: 231.66, Lower: 218.73 | 39.97 (High) | 1.24 (Volatile) | 3.81 (Bullish) |
| AMZN  | 196.31 (Bullish) | 185.42 (Bullish) | 59.48 (Neutral) | Upper: 219.91, Middle: 207.63, Lower: 195.35 | 47.13 (High) | 1.146 (Volatile) | 5.20 (Bullish) |
| GOOGL | 169.16 (Bullish) | 164.83 (Bullish) | 45.71 (Neutral) | Upper: 183.64, Middle: 173.73, Lower: 163.82 | 22.90 (Moderate) | 1.034 (Moderate) | 0.39 (Bullish) |
| META  | 578.49 (Bullish) | 517.67 (Bullish) | 64.56 (Neutral) | Upper: 615.84, Middle: 578.74, Lower: 541.65 | 28.74 (Moderate) | 1.215 (Volatile) | 7.91 (Bullish) |
| MSFT  | 420.63 (Bullish) | 421.19 (Neutral) | 64.18 (Neutral) | Upper: 439.02, Middle: 423.10, Lower: 407.18 | 36.49 (High) | 0.904 (Stable) | 4.14 (Bullish) |
| NVDA  | 137.50 (Bullish) | 113.48 (Bullish) | 47.80 (Neutral) | Upper: 151.86, Middle: 143.11, Lower: 134.35 | 57.11 (High) | 1.657 (Volatile) | 0.92 (Bullish) |
| TSLA  | 281.37 (Bullish) | 218.07 (Bullish) | 73.95 (Overbought) | Upper: 372.71, Middle: 338.43, Lower: 304.14 | 100.95 (Very High) | 2.295 (Very Volatile) | 23.04 (Bullish) |

### Table 2: Analysis and Recommendations

| Asset | Current Trend | Recommendation | Confidence | Elements of Recommendation |
|-------|---------------|----------------|------------|-----------------------------|
| AAPL  | Uptrend | Hold/Buy | High | Strong MACD, Overbought RSI, Above SMA50 & SMA200 |
| AMZN  | Uptrend | Hold/Buy | Medium | Bullish MACD, Above SMA50 & SMA200, Neutral RSI |
| GOOGL | Range (163.82 - 183.64) | Hold | Medium | Neutral RSI, Bullish MACD, Close to Upper Bollinger Band |
| META  | Uptrend | Hold/Buy | High | Strong MACD, Above SMA50 & SMA200, Neutral RSI |
| MSFT  | Range (407.18 - 439.02) | Hold | Medium | Neutral RSI, Bullish MACD, Close to Upper Bollinger Band |
| NVDA  | Uptrend | Hold/Buy | Medium | Bullish MACD, Above SMA50 & SMA200, Neutral RSI |
| TSLA  | Uptrend | Hold/Buy | High | Strong MACD, Overbought RSI, Above SMA50 & SMA200 |

### Summary:
- **AAPL, META, TSLA**: Strong uptrend with high confidence in holding or buying due to bullish indicators and overbought RSI suggesting strong momentum.
- **AMZN, NVDA**: Uptrend with medium confidence; indicators suggest potential for growth but with some caution due to high P/E ratios.
- **GOOGL, MSFT**: Currently in a range; hold recommendation with medium confidence as they are near upper Bollinger Bands and have neutral RSI, indicating potential for breakout or pullback.

These recommendations are based on the current technical indicators and market conditions. Investors should consider their risk tolerance and investment goals before making decisions.

In [4]:
# Function to invoke LLM response based on given prompt
def get_llm_response(llm, prompt):
    """
    Get the response from the language model for a given prompt.

    Parameters:
    llm (): The initialized language model.
    prompt (str): The prompt to send to the language model.

    Returns:
    str: The content of the language model's response.
    """
    response = llm.invoke(prompt)
    return display(Markdown(response.content))


In [ ]:
!pip install openai==1.55.3 langchain-openai==0.2.11

In [41]:
from langchain_openai import ChatOpenAI

chatgpt_api_key = userdata.get('CHATGPT_API_KEY')


# Function to initialize OpenAI LLM model
def initialize_ChatGPT_llm(model, api_key):
    """
    Initialize the ChatOpenAI language model.

    Parameters:
    model (str): The name of the language model to use.
    api_key (str): The API key for accessing the OpenAI service.

    Returns:
    ChatOpenAI: The initialized language model.
    """
    llm = ChatOpenAI(
        model=model,
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key=api_key
    )
    return llm

# Intialize OpenAI LLM
chatgpt_llm = initialize_ChatGPT_llm("gpt-4o", api_key=chatgpt_api_key)


In [ ]:
# Get the response from the LLM based on the provided prompt
get_llm_response(llm = chatgpt_llm, prompt = prompt)

In [ ]:
!pip install langchain-google-genai

In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import auth
auth.authenticate_user()

GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')

# Function to initialize Gemini LLM model
def initialize_ChatGPT_llm(model, api_key):
    """
    Initialize the Gemini AI language model.

    Parameters:
    model (str): The name of the language model to use.
    api_key (str): The API key for accessing the OpenAI service.

    Returns:
    ChatGoogleGenerativeAI: The initialized language model.
    """
    llm = ChatGoogleGenerativeAI(
        model=model,
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key=api_key
    )
    return llm

# Intialize Gemini LLM
gemini_llm = initialize_llm("gemini-pro", api_key=GEMINI_API_KEY)


In [28]:
# Get the response from the LLM based on the provided prompt
get_llm_response(llm = gemini_llm, prompt = prompt)

**Table 1: Key Performance Indicators (KPIs)**

| Asset | SMA50 | SMA200 | RSI | Bollinger Bands | P/E Ratio | Beta | MACD |
|---|---|---|---|---|---|---|---|
| AAPL | NaN | NaN | NaN | NaN | 22.939687 | 1.24 | 0.000000 |
| AMZN | NaN | NaN | NaN | NaN | 18.901709 | 1.146 | 0.000000 |
| GOOGL | NaN | NaN | NaN | NaN | 12.560411 | 1.034 | 0.000000 |
| META | NaN | NaN | NaN | NaN | 5.360453 | 1.215 | 0.000000 |
| MSFT | NaN | NaN | NaN | NaN | 19.820640 | 0.904 | 0.000000 |
| NVDA | NaN | NaN | NaN | NaN | 6.341845 | 1.657 | 0.000000 |
| TSLA | NaN | NaN | NaN | NaN | 47.551911 | 2.295 | 0.000000 |

**Table 2: Recommendations**

| Asset | Current Trend | Recommendation | Confidence | Elements of Recommendation |
|---|---|---|---|---|
| AAPL | Uptrend | Buy | High | The stock is trading above its 50-day and 200-day moving averages, and the RSI is above 50. The MACD is also bullish, with the MACD line above the signal line and the histogram bars increasing. |
| AMZN | Range (204.505000 - 214.771443) | Hold | Medium | The stock is trading within a range, and the RSI is between 30 and 70. The MACD is also neutral, with the MACD line and the signal line close to each other. |
| GOOGL | Uptrend | Buy | High | The stock is trading above its 50-day and 200-day moving averages, and the RSI is above 50. The MACD is also bullish, with the MACD line above the signal line and the histogram bars increasing. |
| META | Range (548.915295 - 592.884711) | Hold | Medium | The stock is trading within a range, and the RSI is between 30 and 70. The MACD is also neutral, with the MACD line and the signal line close to each other. |
| MSFT | Uptrend | Buy | High | The stock is trading above its 50-day and 200-day moving averages, and the RSI is above 50. The MACD is also bullish, with the MACD line above the signal line and the histogram bars increasing. |
| NVDA | Uptrend | Buy | High | The stock is trading above its 50-day and 200-day moving averages, and the RSI is above 50. The MACD is also bullish, with the MACD line above the signal line and the histogram bars increasing. |
| TSLA | Uptrend | Buy | High | The stock is trading above its 50-day and 200-day moving averages, and the RSI is above 50. The MACD is also bullish, with the MACD line above the signal line and the histogram bars increasing. |

In [ ]:
!pip install langchain_mistralai

In [31]:
from langchain_mistralai import ChatMistralAI

MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')

# Function to initialize OpenAI LLM model
def initialize_llm_Mistral(api_key):
    """
    Initialize the ChatOpenAI language model.

    Parameters:
    api_key (str): The API key for accessing the OpenAI service.

    Returns:
    ChatMistralAI: The initialized language model.
    """
    llm = ChatMistralAI(
        model="mistral-large-latest",
        temperature=0,
        max_retries=2,
        api_key=api_key
    )

    return llm

# Intialize OpenAI LLM
mistral_llm = initialize_llm_Mistral(api_key=MISTRAL_API_KEY)


In [32]:
# Get the response from the LLM based on the provided prompt
get_llm_response(llm = mistral_llm, prompt = prompt)

### Executive Summary

#### Table 1: Key Performance Indicators (KPIs)

| Asset | SMA50 (50-Day Simple Moving Average) | SMA200 (200-Day Simple Moving Average) | RSI (Relative Strength Index) | P/E Ratio | Beta | MACD | Signal Line | MACD Histogram |
|-------|-----------------------------------|--------------------------------------|-----------------------------|-----------|------|------|-------------|----------------|
| AAPL  | 229.587607 (Uptrend)                | 206.618662 (Uptrend)                    | 85.009779 (Overbought)        | 39.973683  | 1.24 | 3.811727 (Bullish) | 2.213700 (Bullish) | 1.598028 (Bullish) |
| AMZN  | 196.312401 (Uptrend)                | 185.42470 (Uptrend)                    | 59.477540 (Neutral)          | 47.126069  | 1.146 | 5.197577 (Bullish) | 4.099027 (Bullish) | 1.098551 (Bullish) |
| GOOGL | 169.162000 (Uptrend)                | 164.829085 (Uptrend)                   | 45.711782 (Neutral)          | 22.896552  | 1.034 | 0.388114 (Bullish) | 0.655361 (Bullish) | -0.267248 (Bearish) |
| META  | 578.487404 (Uptrend)                | 517.669541 (Uptrend)                   | 64.564058 (Neutral)          | 28.736668  | 1.215 | 7.913652 (Bullish) | 2.694766 (Bullish) | 5.218886 (Bullish) |
| MSFT  | 420.627208 (Uptrend)                | 421.189907 (Uptrend)                   | 64.176675 (Neutral)          | 36.489695  | 0.904 | 4.143024 (Bullish) | 1.633880 (Bullish) | 2.509144 (Bullish) |
| NVDA  | 137.504725 (Uptrend)                | 113.477836 (Uptrend)                   | 47.797764 (Neutral)          | 57.110235  | 1.657 | 0.924158 (Bullish) | 1.193883 (Bullish) | -0.269725 (Bearish) |
| TSLA  | 281.371600 (Uptrend)                | 218.06770 (Uptrend)                    | 73.954501 (Overbought)       | 100.953549 | 2.295 | 23.039054 (Bullish) | 23.191225 (Bullish) | -0.152171 (Bearish) |

#### Table 2: Recommendations

| Asset | Current Trend                     | Recommendation                        | Confidence | Elements of Recommendation                                                                 |
|-------|----------------------------------|--------------------------------------|------------|--------------------------------------------------------------------------------------------|
| AAPL  | Uptrend                           | Buy                                  | High       | Strong uptrend indicated by SMA50 and SMA200. Overbought RSI suggests potential correction, but MACD and Signal Line are bullish. |
| AMZN  | Uptrend                           | Buy                                  | High       | Consistent uptrend with neutral RSI. MACD and Signal Line are bullish, indicating strong momentum.                             |
| GOOGL | Uptrend                           | Hold                                 | Moderate   | Uptrend with neutral RSI. MACD is bullish, but Signal Line and MACD Histogram are bearish, suggesting potential reversal.      |
| META  | Uptrend                           | Buy                                  | High       | Strong uptrend with neutral RSI. MACD and Signal Line are bullish, indicating strong momentum.                               |
| MSFT  | Uptrend                           | Buy                                  | High       | Consistent uptrend with neutral RSI. MACD and Signal Line are bullish, indicating strong momentum.                           |
| NVDA  | Uptrend                           | Hold                                 | Moderate   | Uptrend with neutral RSI. MACD is bullish, but Signal Line and MACD Histogram are bearish, suggesting potential reversal.      |
| TSLA  | Uptrend                           | Hold                                 | Moderate   | Strong uptrend with overbought RSI. MACD and Signal Line are bullish, but MACD Histogram is bearish, suggesting potential reversal. |

### Recommendations Summary

- **AAPL**: Strong buy recommendation due to consistent uptrend and bullish MACD indicators.
- **AMZN**: Strong buy recommendation due to consistent uptrend and bullish MACD indicators.
- **GOOGL**: Hold recommendation due to mixed signals from MACD and Signal Line.
- **META**: Strong buy recommendation due to consistent uptrend and bullish MACD indicators.
- **MSFT**: Strong buy recommendation due to consistent uptrend and bullish MACD indicators.
- **NVDA**: Hold recommendation due to mixed signals from MACD and Signal Line.
- **TSLA**: Hold recommendation due to overbought RSI and mixed signals from MACD and Signal Line.

### Conclusion

The analysis suggests a bullish outlook for most assets, with strong uptrends and bullish MACD indicators. However, caution is advised for GOOGL, NVDA, and TSLA due to mixed signals and potential reversals. Investors should monitor these assets closely and consider diversifying their portfolios to mitigate risk.

In [33]:
!pip install langchain-anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 7.8 MB/s eta 0:00:00


In [38]:
from langchain_anthropic import ChatAnthropic

ANTHROPIC_API_KEY=userdata.get('ANTHROPIC_API_KEY')

# Function to initialize OpenAI LLM model
def initialize_llm_Anthropic(api_key):
    """
    Initialize the ChatOpenAI language model.

    Parameters:
    api_key (str): The API key for accessing the OpenAI service.

    Returns:
    ChatMistralAI: The initialized language model.
    """
    llm = ChatAnthropic(
        model='claude-3-opus-20240229',
        temperature=0,
        max_retries=2,
        api_key=api_key
    )

    return llm

# Intialize OpenAI LLM
Anthropic_llm = initialize_llm_Anthropic(api_key=ANTHROPIC_API_KEY)


In [39]:
get_llm_response(llm = Anthropic_llm, prompt = prompt)

Here is the executive summary with recommendations for each asset, presented in two tables:

Table 1: KPIs Recap

| Asset | SMA50      | SMA200     | RSI                   | Bollinger Bands                      | P/E Ratio            | Beta  | MACD                                                 |
|-------|------------|------------|----------------------|--------------------------------------|---------------------|-------|------------------------------------------------------|
| AAPL  | Uptrend    | Uptrend    | Overbought (82.96)   | Price near upper band                | Overvalued (39.97)  | 1.24  | Bullish crossover, positive divergence              |
| AMZN  | Uptrend    | Uptrend    | Neutral (59.48)      | Price within bands                   | Overvalued (47.13)  | 1.146 | Bullish crossover, positive divergence              |
| GOOGL | Uptrend    | Uptrend    | Neutral (45.71)      | Price near middle band               | Overvalued (22.90)  | 1.034 | Bearish crossover, negative divergence              |
| META  | Uptrend    | Uptrend    | Neutral (64.56)      | Price near upper band                | Overvalued (28.74)  | 1.215 | Bullish crossover, positive divergence              |
| MSFT  | Range-bound| Range-bound| Neutral (64.18)      | Price oscillating within bands       | Overvalued (36.49)  | 0.904 | Bullish crossover, positive divergence              |
| NVDA  | Uptrend    | Uptrend    | Neutral (47.80)      | Price near middle band               | Overvalued (57.11)  | 1.657 | Bearish crossover, negative divergence              |
| TSLA  | Uptrend    | Uptrend    | Overbought (73.95)   | Price near middle band               | Overvalued (100.95) | 2.295 | Bullish crossover, positive divergence              |

Table 2: Recommendations

| Asset | Current Trend      | Recommendation | Confidence | Elements                                                                 |
|-------|------------------- |----------------|------------|-------------------------------------------------------------------------|
| AAPL  | Uptrend            | Hold           | Medium     | Overbought RSI, overvalued P/E, bullish MACD                            |
| AMZN  | Uptrend            | Buy            | High       | Neutral RSI, bullish MACD, price within Bollinger Bands                 |
| GOOGL | Uptrend            | Sell           | Medium     | Bearish MACD, overvalued P/E                                            |
| META  | Uptrend            | Buy            | High       | Bullish MACD, price near upper Bollinger Band                           |
| MSFT  | Range ($407 - $439)| Hold           | Medium     | Neutral RSI, overvalued P/E, price oscillating within Bollinger Bands   |
| NVDA  | Uptrend            | Sell           | Medium     | Bearish MACD, highly overvalued P/E                                     |
| TSLA  | Uptrend            | Hold           | Low        | Overbought RSI, highly overvalued P/E, bullish MACD                     |

In summary, AMZN and META show the strongest buy signals based on their bullish MACD, neutral RSI, and price action within Bollinger Bands. GOOGL and NVDA display sell signals due to bearish MACD and overvalued P/E ratios. AAPL, MSFT, and TSLA are holds as they present mixed signals, with overbought RSI and overvalued P/E ratios counterbalanced by bullish MACD indicators. The confidence in these recommendations varies based on the strength and alignment of the various technical and fundamental factors.